Basé sur le noyau Keras CNN avec une précision supérieure à 99%

In [29]:
import numpy as np

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization

In [ ]:
train_file = "C:/Users/aaron/OneDrive/Bureau/Digit_Recognizer/data/train.csv"
test_file = "C:/Users/aaron/OneDrive/Bureau/Digit_Recognizer/data/test.csv"
output_file = "C:/Users/aaron/OneDrive/Bureau/Digit_Recognizer/data/sample_submission.csv"

In [ ]:
mnist_dataset = np.loadtxt(train_file, skiprows=1, dtype='int', delimiter=',')

In [10]:
val_split = 0.125
n_raw = mnist_dataset.shape[0]
n_val = int(n_raw * val_split + 0.5)
n_train = n_raw - n_val

np.random.shuffle(mnist_dataset)
x_val, x_train = mnist_dataset[:n_val,1:], mnist_dataset[n_val:,1:]
y_val, y_train = mnist_dataset[:n_val,0], mnist_dataset[n_val:,0]

x_train = x_train.astype("float32")/255.0
x_val = x_val.astype("float32")/255.0
y_train = np_utils.to_categorical(y_train)
y_val = np_utils.to_categorical(y_val)

n_classes = y_train.shape[1]
x_train = x_train.reshape(n_train, 28, 28, 1)
x_val = x_val.reshape(n_val, 28, 28, 1)

In [11]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (3, 3), input_shape = (28, 28, 1), 
                 padding='same', kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(Conv2D(filters = 64, kernel_size = (3, 3), 
                 padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters = 128, kernel_size = (3, 3),
                 padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters = 128, kernel_size = (3, 3),
                 padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(Dropout(0.5))

model.add(Dense(256, kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(Dropout(0.5))

model.add(Dense(10, kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [12]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 28, 28, 64)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)       

Cette fonction Keras simplifie l'augmentation, c'est-à-dire qu'elle modifie de manière aléatoire l'entrée pour la formation pour éviter le surapprentissage.

In [14]:
datagen = ImageDataGenerator(zoom_range = 0.1,
                            height_shift_range = 0.1,
                            width_shift_range = 0.1,
                            rotation_range = 20)

In [18]:
model.compile(loss='categorical_crossentropy', optimizer = Adam(lr=5e-4), metrics = ["accuracy"])

EarlyStopping n'est important que si nous prévoyons d'exécuter de nombreuses époques. ModelCheckpoint est plus utile ici, car vnous pouvons reprendre la formation après un redémarrage du noyau: exécutez simplement

>model.load_weights ('' mnist.h5 '')

In [19]:
callbacks = [EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=2, mode='auto'),
            ModelCheckpoint('mnist.h5', monitor='val_loss', save_best_only=True, verbose=0)]

Formation de modèle. Notez que la perte de formation est initialement beaucoup plus importante que la perte de validation, en raison de l'abandon.

In [22]:
read_model = False
if read_model:
    if os.path.isfile("C:/Users/aaron/OneDrive/Bureau/Digit_Recognizer/output/model.h5"):
        model = load_model("C:/Users/aaron/OneDrive/Bureau/Digit_Recognizer/output/model.h5")

In [21]:
hist = model.fit_generator(datagen.flow(x_train, y_train, batch_size = 2048),
                           steps_per_epoch = n_train/2048, 
                           epochs = 10, 
                           verbose = 1,  
                           validation_data = (x_val, y_val),
                           callbacks = callbacks)

Epoch 1/10
18/17 [==============================] - 192s 11s/step - loss: 2.8184 - accuracy: 0.3811 - val_loss: 2.2421 - val_accuracy: 0.3878
Epoch 2/10
18/17 [==============================] - 189s 11s/step - loss: 0.9464 - accuracy: 0.7196 - val_loss: 1.5901 - val_accuracy: 0.4966
Epoch 3/10
18/17 [==============================] - 189s 11s/step - loss: 0.5736 - accuracy: 0.8305 - val_loss: 0.8703 - val_accuracy: 0.7143
Epoch 4/10
18/17 [==============================] - 193s 11s/step - loss: 0.4143 - accuracy: 0.8770 - val_loss: 0.4270 - val_accuracy: 0.8556
Epoch 5/10
18/17 [==============================] - 191s 11s/step - loss: 0.3418 - accuracy: 0.8980 - val_loss: 0.3265 - val_accuracy: 0.8947
Epoch 6/10
18/17 [==============================] - 192s 11s/step - loss: 0.2866 - accuracy: 0.9159 - val_loss: 0.2206 - val_accuracy: 0.9299
Epoch 7/10
18/17 [==============================] - 192s 11s/step - loss: 0.2522 - accuracy: 0.9264 - val_loss: 0.2526 - val_accuracy: 0.9200
Epoch 

In [23]:
from keras.models import load_model

# save json
json_model = model.to_json()
with open("C:/Users/aaron/OneDrive/Bureau/Digit_Recognizer/output/model.json", "w") as json_file:
    json_file.write(json_model)
# save model
model.save("C:/Users/aaron/OneDrive/Bureau/Digit_Recognizer/output/model.h5")

In [24]:
mnist_testset = np.loadtxt(test_file, skiprows=1, dtype='int', delimiter=',')
x_test = mnist_testset.astype("float32")/255.0
n_test = x_test.shape[0]
x_test = x_test.reshape(n_test, 28, 28, 1)

In [25]:
y_test = model.predict(x_test, batch_size=128)

y_test se compose de probabilités de classe, je sélectionne maintenant la classe avec la probabilité la plus élevée

In [26]:
y_index = np.argmax(y_test,axis=1)

In [27]:
with open(output_file, 'w') as f :
    f.write('ImageId,Label\n')
    for i in range(0,n_test) :
        f.write("".join([str(i+1),',',str(y_index[i]),'\n']))

# Kaggle Score 0,99,17 (99.17%)

> Classement Kaggle
 >> aaron djoko : 666